In [27]:
import pandas as pd
import numpy as np
import os
import plotly.express as px
from prophet import Prophet


In [14]:
##import 2015-2022 Denver's permitting data
path = '/Users/shutingli/Desktop/shuting/Shuting\'s Job/Community Dreams/github_CDF/Community-Dreams-Program/Data/Denver'

files = [file for file in os.listdir(path) if file.endswith('.xlsx') or file.endswith('.xls')]

dataframes = []

for file in files:
    full_path = os.path.join(path, file)
    df = pd.read_excel(full_path, parse_dates=['Date Issued'])
    # remove the last row
    df = df.drop(df.tail(1).index)
        
    dataframes.append(df)

Denver = pd.concat(dataframes, ignore_index=True)

##remove NA rows
Denver.sort_values('Date Issued', inplace=True)
Denver = Denver.dropna(axis=0,how='all')


WARNING *** file size (10542105) not 512 + multiple of sector size (512)
WARNING *** file size (20294169) not 512 + multiple of sector size (512)
WARNING *** file size (19726361) not 512 + multiple of sector size (512)
WARNING *** file size (17245721) not 512 + multiple of sector size (512)
WARNING *** file size (16876569) not 512 + multiple of sector size (512)
WARNING *** file size (8681497) not 512 + multiple of sector size (512)
WARNING *** file size (16690201) not 512 + multiple of sector size (512)


In [32]:
##permit fee on a daily basis from 2015~2023
Valuation = Denver.groupby(Denver['Date Issued'].dt.date)['Valuation'].sum()

# To get this back as a DataFrame:
Valuation_df = Valuation.reset_index()

Valuation_df

,Date Issued,Valuation
0,2015-01-02,550674
1,2015-01-05,2055924
2,2015-01-06,6296962
3,2015-01-07,6643312
4,2015-01-08,6283250
...,...,...
2647,2023-07-08,467922
2648,2023-07-09,284682
2649,2023-07-10,6488908.85
2650,2023-07-11,12655347.4


In [36]:
fig = px.line(Valuation_df, x='Date Issued', y='Valuation')
fig.show()

##use data from 2018, because we lose data in later half year of 2017
Valuation_df['Date Issued'] = pd.to_datetime(Valuation_df['Date Issued'], format='%Y-%m-%d') 
Valuation_df_filtered = Valuation_df[Valuation_df['Date Issued'].dt.year.isin([2018, 2019, 2020, 2021, 2022, 2023])]

fig = px.line(Valuation_df_filtered, x='Date Issued', y='Valuation')
fig.show()


## Fit ARIMA using Prophet

In [37]:
# Rename the columns to meet Prophet's requirements
Valuation_df_filtered = Valuation_df_filtered.rename(columns={"Date Issued": "ds", "Valuation": "y"})

# Initialize the Prophet model
model = Prophet()

# Fit the model with your dataframe
model.fit(Valuation_df_filtered)

# Create a dataframe to hold predictions
future = model.make_future_dataframe(periods=365)  # for 1 year

# Generate predictions
forecast = model.predict(future)


19:10:07 - cmdstanpy - INFO - Chain [1] start processing
19:10:07 - cmdstanpy - INFO - Chain [1] done processing


In [38]:
from prophet.plot import plot_plotly, plot_components_plotly

# Plot the forecast
fig1 = plot_plotly(model, forecast)
fig1.show()

# Plot the components (trend, yearly seasonality, and weekly seasonality)
fig2 = plot_components_plotly(model, forecast)
fig2.show()

### Monthly & Quarterly prediciton

In [45]:
Valuation_forecast = forecast.loc[:,['ds', 'yhat']]
Valuation_forecast

,ds,yhat
0,2018-01-01,8.095157e+06
1,2018-01-02,1.210755e+07
2,2018-01-03,1.150841e+07
3,2018-01-04,1.127760e+07
4,2018-01-05,1.040516e+07
...,...,...
2366,2024-07-07,1.406212e+06
2367,2024-07-08,1.559841e+07
2368,2024-07-09,1.965845e+07
2369,2024-07-10,1.902190e+07


In [47]:
# Set 'ds' as the index
Valuation_forecast.set_index('ds', inplace=True)

# Resample and sum up the 'yhat' values on a monthly basis
monthly_forecast = Valuation_forecast['yhat'].resample('M').sum()
monthly_forecast_after_July_2023 = monthly_forecast[monthly_forecast.index >= '2023-07']

print(monthly_forecast_after_July_2023)


ds
2023-07-31    3.731122e+08
2023-08-31    4.549360e+08
2023-09-30    4.835367e+08
2023-10-31    4.101314e+08
2023-11-30    4.829701e+08
2023-12-31    4.853548e+08
2024-01-31    4.962247e+08
2024-02-29    5.133421e+08
2024-03-31    4.878453e+08
2024-04-30    4.587236e+08
2024-05-31    4.525799e+08
2024-06-30    4.365131e+08
2024-07-31    1.673979e+08
Freq: M, Name: yhat, dtype: float64


In [48]:
# Resample and sum up the 'yhat' values on a quarterly basis
quarterly_forecast = Valuation_forecast['yhat'].resample('Q').sum()
quarterly_forecast_after_July_2023 = quarterly_forecast[quarterly_forecast.index >= '2023-07']

print(quarterly_forecast_after_July_2023)

ds
2023-09-30    1.311585e+09
2023-12-31    1.378456e+09
2024-03-31    1.497412e+09
2024-06-30    1.347817e+09
2024-09-30    1.673979e+08
Freq: Q-DEC, Name: yhat, dtype: float64


#### 2023-08 as example

In [62]:
##prediction of August permitting projects' valuation
August = monthly_forecast_after_July_2023.loc['2023-08-31']
August

454936034.3997953

In [63]:
df = Denver.dropna(subset=['Valuation'], how='any', axis=0)

# ensure 'Date Issued' is datetime
df['Date Issued'] = pd.to_datetime(df['Date Issued'])
avg_value = np.mean(df.loc[df['Date Issued'].dt.year.isin([2022,2023]), 'Valuation'])
avg_value

/var/folders/_5/_6xnqpw50q72w4mx698g23kw0000gn/T/ipykernel_1898/806937017.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



85477.46482880584

In [64]:
((August * 0.5) / avg_value) * 3

7983.438125667519

Given the assumption that: 

1/3 individuals will submit the appication

50% of applications move into project phase

Average project value in recently 2 years is 85477

We can make a prediction that in August, there will be 7983 customers in Denver have demand for house upgrading. But only 1/3 of them will submit applications, so we can capture these part of customer and help them to go through upgrading process.